In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datashader as ds
import plotnine as pln
import pickle as pkl

from scipy import stats

from sklearn.preprocessing import OneHotEncoder

import datashader.transfer_functions as tf

# Import the file and preprocess it

In [ ]:
data = pd.read_csv('./ML case/data.csv', parse_dates=['date_in'], dtype={'agency_rating': np.int64})

`agancy_rating`, the only ordinal variable will be treated as a numeric one for the sake of this classification.

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data['year'] = data['date_in'].dt.year
data['month'] = data['date_in'].dt.month
data['day'] = data['date_in'].dt.day
data['weekday'] = data['date_in'].dt.weekday

In [ ]:
data['house_pk'].value_counts().plot(kind='hist')

In [ ]:
data[['agency_id', 'agency_rating']].pivot_table(index='agency_id', columns='agency_rating', aggfunc=len)

In [ ]:
# we want to preserve agency_id for now
data = pd.concat([data['agency_id'], pd.get_dummies(data, columns=['agency_id'], drop_first=True)], axis=1, sort=False)

# Exploratory data anlysis (EDA)

First, we will discover the relationship between the target variable and the features.

### Plot y's (price's) distribution

In [ ]:
sns.distplot(data['price'])

As the distribution of price is highly non-normal, it will be more reasonable to work with log-normalized data.

In [ ]:
data['price'] = np.log10(data['price'])

In [ ]:
sns.distplot(data['price'])

### Plot mean price over time and add some context to it with all of the observations on one plot

In [ ]:
holidays = pd.DataFrame({'start': ['2016-07-01', '2017-07-01', '2018-07-01'], 'finish': ['2016-08-31', '2017-08-31', '2018-08-31'], 
                         'ymin': [-np.inf, -np.inf, -np.inf], 'ymax': [np.inf, np.inf, np.inf]}, )
holidays['start'] = pd.to_datetime(holidays['start'])
holidays['finish'] = pd.to_datetime(holidays['finish'])

christmas = pd.DataFrame({'start': ['2016-12-20', '2017-12-20', '2018-12-20'], 'finish': ['2017-01-15', '2018-01-15', '2019-01-15'], 
                         'ymin': [-np.inf, -np.inf, -np.inf], 'ymax': [np.inf, np.inf, np.inf]}, )
christmas['start'] = pd.to_datetime(christmas['start'])
christmas['finish'] = pd.to_datetime(christmas['finish'])

In [ ]:
(pln.ggplot(pln.aes('date_in', 'price', group='factor(house_pk)'), data=data)
 + pln.geom_line(color='gray', alpha=0.3, size=0.5)
 + pln.geom_line(pln.aes('date_in', 'price'), data.groupby('date_in')['price'].agg('mean').to_frame().reset_index(drop=False), size=3, inherit_aes=False)
 + pln.geom_rect(pln.aes(xmin='start', xmax='finish', ymin='ymin', ymax='ymax'), holidays, fill='green', alpha=0.1, inherit_aes=False)
 + pln.geom_rect(pln.aes(xmin='start', xmax='finish', ymin='ymin', ymax='ymax'), christmas, fill='red', alpha=0.1, inherit_aes=False)
 + pln.labels.xlab('Date')
 + pln.labels.ylab('Price')
 + pln.theme_bw()
 + pln.theme(axis_text_x=pln.element_text(rotation=90, hjust=0.5)))

### Categorical variables

It pays off to plot the distribution with respect to the year even if it cannot be used as an explanatory variable.

In [ ]:
cat_variables = ['agency_id', 'apartment', 'indoor_pool', 'spa', 'internet', 'pets_allowed', 'water_view', 'fire_stove', 'agency_rating', 'year', 
                 'month', 'weekday']

In [ ]:
n_rows = len(cat_variables) // 2 + len(cat_variables) % 2
n_cols = 2
fig, ax = plt.subplots(ncols=n_cols, nrows=n_rows, sharey=True, figsize=(18, 50))

for i, var in enumerate(cat_variables):
    var_name = var.upper().replace('_', ' ')
    sns.boxplot(data=data, x=var, y='price', ax=ax[i // 2, i % 2])
    # please mind that the data has been already log-transformed
    _, p = stats.f_oneway(*[data['price'].loc[x[1][var].index].values for x in data[var].to_frame().groupby(var, squeeze=True)])
    ax[i // 2, i % 2].set_title(f'{var_name}, one-way ANOVA p-value: {np.round(p, 4)}')
    ax[i // 2, i % 2].set_xlabel(var_name)
    ax[i // 2, i % 2].set_ylabel('PRICE')
    
# remove unused axes
if i+1 != n_rows*n_cols:
    ax.flat[-1].set_visible(False)

### Continuous variables

In [ ]:
con_variables = ['dis_water_real', 'dis_shopping', 'no_bedrooms', 'max_persons', 'house_size', 'land_size', 'build_year', 'renovation_year']

Apparently, `no_bedrooms` could be treated as a categorical variable due to the low number of values it takes.

In [ ]:
n_rows = len(con_variables) // 2 + len(con_variables) % 2
n_cols = 2

fig, ax = plt.subplots(ncols=n_cols, nrows=n_rows, sharey=True, figsize=(18, 30))

for i, var in enumerate(con_variables):
    var_name = var.upper().replace('_', ' ')
    sns.scatterplot(data=data, x=var, y='price', ax=ax[i // 2, i % 2])
    ax[i // 2, i % 2].set_title(var_name)
    ax[i // 2, i % 2].set_xlabel(var_name)
    ax[i // 2, i % 2].set_ylabel('PRICE')
    
# remove unused axes
if i+1 != n_rows*n_cols:
    ax.flat[-1].set_visible(False)

### Correlation between features

In [ ]:
all_variables = cat_variables + con_variables

In [ ]:
palette = sns.xkcd_palette(colors=["windows blue", "amber"])
col_mapping = [palette[0] if var in cat_variables else palette[1] for var in all_variables]

In [ ]:
# legend for the rectangles on the plot
sns.palplot(palette)
for i, val in enumerate(['categorical', 'continuous']):
    plt.text(i, 0, val, horizontalalignment='center', verticalalignment='center', rotation=45)
plt.show()

In [ ]:
sns.clustermap(data[all_variables].corr(method='pearson'), method='ward', figsize=(15, 15), row_cluster=True, col_cluster=True, col_colors=col_mapping)

# Regression model, solely on tabular data, to predict price

In [ ]:
# no weekday
X_features = ['dis_water_real', 'dis_shopping', 'no_bedrooms', 'max_persons', 'house_size', 'land_size', 'build_year', 'renovation_year', 'apartment', 
              'indoor_pool', 'spa', 'internet', 'pets_allowed', 'water_view', 'fire_stove', 'agency_rating', 'year', 'month', 'day', 'agency_id_121', 
              'agency_id_130', 'agency_id_160']

### Prepare validation set

Validation set consists of 15% of observations for a given house_pk within a given year and month.

In [ ]:
random_state = np.random.RandomState(37676373)

In [ ]:
validation_set = data.groupby(['house_pk', 'year', 'month']).apply(lambda gr: gr.sample(frac=0.15, random_state=random_state)).reset_index(drop=True)

In [ ]:
training_set = pd.merge(data, validation_set, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [ ]:
assert validation_set.shape[0] > 0 and training_set.shape[0] > 0 and validation_set.shape[0] + training_set.shape[0] == data.shape[0]

### Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
rf_model = RandomForestRegressor(n_estimators=250, criterion='mse', max_features='sqrt', n_jobs=3, random_state=random_state)

rf_grid = {'max_depth': [20, 22, 25, 27, 30]}  # the grid was chosen after some initial trials

rf_cv = GridSearchCV(rf_model, rf_grid, scoring='neg_mean_squared_error', 
                     cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=random_state), 
                     n_jobs=2, return_train_score=False)

In [ ]:
rf_cv.fit(training_set[X_features], training_set['price'])

In [ ]:
# pkl.dump(rf_cv, open('rf_model.pkl', 'wb'))

In [ ]:
rf_cv = pkl.load(open('rf_model.pkl', 'rb'))

In [ ]:
rf_cv.best_params_

In [ ]:
rf_predictions = rf_cv.predict(validation_set[X_features])

### LightGBM

In [ ]:
import lightgbm as lgbm
from skopt import BayesSearchCV
from skopt.space import Real, Integer

In [ ]:
lgbm_model = lgbm.LGBMRegressor(n_jobs=3, num_leaves=2**10, random_state=37676373, importance_type='gain', )

lgbm_grid = {'n_estimators': Integer(50, 500),
             'learning_rate': Real(0.01, 0.35, 'uniform'),
             'reg_alpha': Real(1e-6, 1, 'log-uniform'), 
             'reg_lambda': Real(1e-6, 1, 'log-uniform'), 
             'gamma': Real(0, 1, 'uniform'), 
             'subsample': Real(0.5, 1, 'uniform'), 
             'colsample_bytree': Real(0.5, 1, 'uniform'),
             'max_depth': Integer(10, 30)
            }

lgbm_cv = BayesSearchCV(lgbm_model, lgbm_grid, n_iter=25, scoring='neg_mean_squared_error', refit=True, n_points=1,
                        cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=random_state),
                        n_jobs=3, random_state=random_state)

In [ ]:
lgbm_cv.fit(training_set[X_features], training_set['price'])

In [ ]:
pkl.dump(lgbm_cv, open('lgbm_model.pkl', 'wb'))

In [ ]:
lgbm_cv = pkl.load(open('lgbm_cv.pkl', 'rb'))

In [ ]:
lgbm_cv.best_params_

In [ ]:
lgbm_predictions = lgbm_cv.predict(validation_set[X_features])

### Compare  both models side-by-side

In [ ]:
fig, ax = plt.subplots(1, 2, True, True, figsize=(20, 7))
# the two lines below are for common labels for both plots
fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)

ax[0].plot(validation_set['price'], rf_predictions, color='green', marker='x', linewidth=0)
ax[0].set_title(f'Random forest predictions - MSE: {np.round(mean_squared_error(validation_set["price"], rf_predictions), 5)}',
                fontdict={'fontsize': 20, 'fontweight': 'bold'})

ax[1].plot(validation_set['price'], lgbm_predictions, color='blue', marker='x', linewidth=0)
ax[1].set_title(f'LightGBM predictions - MSE: {np.round(mean_squared_error(validation_set["price"], lgbm_predictions), 5)}',
                fontdict={'fontsize': 20, 'fontweight': 'bold'})

plt.xlabel('log10(Real price)')
plt.ylabel('log10(Predicted price)')
plt.tight_layout()

plt.show()